In [1]:
import pandas as pd
import statistics as st

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
FIVE, FOUR, THREE, TWO, ONE, ZERO = 5, 4, 3, 2, 1, 0

data_table = pd.DataFrame(columns=["league_id", "week", "hour", "minute", "HT", "AT", "HCS", "ACS"])

In [4]:

def txt_reader(_path):
    file = open(_path, 'r')  # Open the file in read mode
    contents = file.read()  # Read the contents of the file
    file.close()  # Close the file

    return contents

def table_creator(contents):
    
    # data_table = pd.DataFrame(columns=["league_id", "week", "hour", "minute", "HT", "AT", "HCS", "ACS"])

    record_list = []
    weeks = contents.split("WEEK")
    pos = weeks[0].index("League")
    league_no = int(weeks[0][(pos + 7): (pos + 11)])
    for val in weeks[1:]:
        scores = val.split("\n")[1:-1]
        for score in scores:
            temp_dict = {
                "league_id": league_no,
                "week": int((val[:3]).strip()),
                "hour": int(val.split("\n")[0][-8:-6]),
                "minute": int(val.split("\n")[0][-5:-3]),

                "HT": score[0:3],
                "AT": score[8:11],
                "HCS": int(score[4]),
                "ACS": int(score[6])
            }

            # data._append(temp_dict, ignore_index=True)
            record_list.append(temp_dict)
    data = pd.DataFrame(record_list)

    return data

In [14]:
txt_reader("sample_4.txt")

'\nlogo\nLeague 1463\t\nResultsLeague Table\nWEEK 1   21:32:00\nASV\t1-0\tTOT\nFOR\t0-1\tCHE\nLIV\t1-2\tBUR\nMNU\t0-1\tCRY\nFUL\t0-1\tBRI\nEVE\t0-1\tLUT\nBOU\t1-1\tNWC\nSHU\t0-3\tARS\nBRN\t2-2\tWHU\nWOL\t0-2\tMNC\nWEEK 2   21:34:00\nNWC\t0-1\tBRI\nCHE\t0-0\tASV\nTOT\t0-3\tMNU\nARS\t2-0\tFUL\nLUT\t3-0\tSHU\nBOU\t2-1\tBRN\nBUR\t1-0\tEVE\nWHU\t1-2\tFOR\nCRY\t3-0\tWOL\nMNC\t3-1\tLIV\nWEEK 3   21:36:00\nEVE\t0-0\tMNC\nBRI\t1-3\tARS\nFUL\t2-2\tLUT\nWOL\t0-1\tTOT\nFOR\t0-0\tBOU\nLIV\t1-1\tCRY\nMNU\t1-0\tCHE\nSHU\t1-1\tBUR\nBRN\t1-1\tNWC\nASV\t1-0\tWHU\nWEEK 4   21:38:00\nCHE\t2-0\tWOL\nBOU\t1-0\tASV\nBRN\t0-0\tFOR\nMNC\t3-0\tSHU\nLUT\t0-3\tBRI\nBUR\t2-1\tFUL\nCRY\t0-2\tEVE\nNWC\t4-0\tARS\nWHU\t1-1\tMNU\nTOT\t1-5\tLIV\n'

In [15]:
record = txt_reader("sample_4.txt")
df = table_creator(record)
df

,league_id,week,hour,minute,HT,AT,HCS,ACS
0,1463,1,21,32,ASV,TOT,1,0
1,1463,1,21,32,FOR,CHE,0,1
2,1463,1,21,32,LIV,BUR,1,2
3,1463,1,21,32,MNU,CRY,0,1
4,1463,1,21,32,FUL,BRI,0,1
5,1463,1,21,32,EVE,LUT,0,1
6,1463,1,21,32,BOU,NWC,1,1
7,1463,1,21,32,SHU,ARS,0,3
8,1463,1,21,32,BRN,WHU,2,2
9,1463,1,21,32,WOL,MNC,0,2


In [6]:
path = "league_data"
records = ["L_6095", "L_6097", "L_6099"]
df_temp = []
for record in records:
    text = txt_reader(f"league_data/{record}.txt")
    df_temp.append(table_creator(text))

multiple_record = pd.concat(df_temp, axis=0)
multiple_record

,league_id,week,hour,minute,HT,AT,HCS,ACS
0,6095,1,12,52,EVE,MNC,1,2
1,6095,1,12,52,BRN,TOT,0,1
2,6095,1,12,52,BRI,FUL,1,0
3,6095,1,12,52,LEI,WHU,2,1
4,6095,1,12,52,WOL,ARS,0,0
...,...,...,...,...,...,...,...,...
375,6099,38,19,10,CRY,FUL,2,2
376,6099,38,19,10,WHU,EVE,0,0
377,6099,38,19,10,ASV,LEI,1,3
378,6099,38,19,10,MNC,CHE,1,2


In [7]:
teams = ['FOR', 'MNC', 'ASV', 'TOT', 'EVE', 'CHE', 'BRN', 'WHU', 'ARS', 'FUL', 
         'NWC', 'BOU', 'LEI', 'LIV', 'WOL', 'MNU', 'LEE', 'SOU', 'BRI', 'CRY']

class Team:
    def __init__(self, name):
        self.name = name
        self.goals_for = 0
        self.goals_against = 0
        self.goal_difference = 0
        self.points = 0
        self.position = 0

team_objects = [Team(team) for team in teams]

def update_table(team_1, team_1_score, team_2, team_2_score):
    team1 = next(team for team in team_objects if team.name == team_1)
    team2 = next(team for team in team_objects if team.name == team_2)

    team1.goals_for = team_1_score
    team1.goals_against = team_2_score
    team1.goal_difference = team_1_score - team_2_score
    team1.points = 3 if team_1_score > team_2_score else 1 if team_1_score == team_2_score else 0

    team2.goals_for = team_2_score
    team2.goals_against = team_1_score
    team2.goal_difference = team_2_score - team_1_score
    team2.points = 3 if team_2_score > team_1_score else 1 if team_2_score == team_1_score else 0

    table = sorted(team_objects, key=lambda x: (-x.points, -x.goal_difference, -x.goals_for, x.name))

    for i, team in enumerate(table):
        team.position = i + 1

    return table

In [ ]:
for i, row in df.iterrows():
    home_team, home_score, away_team, away_score = row["HT"], row["HCS"], row["AT"], row["ACS"]
    table = update_table(home_team, home_score, away_team, away_score)
    

In [33]:
# Example usage
table = update_table('BOU', 2, 'ASV', 1)

# Print the table
for team in table:
    print("Team:", team.name)
    print("Position:", team.position)
    print("Points:", team.points)
    print("Goal Difference:", team.goal_difference)
    print("Goals For:", team.goals_for)
    print("Goals Against:", team.goals_against)
    print()



Team: MNC
Position: 1
Points: 3
Goal Difference: 2
Goals For: 2
Goals Against: 0

Team: BOU
Position: 2
Points: 3
Goal Difference: 1
Goals For: 2
Goals Against: 1

Team: BRI
Position: 3
Points: 0
Goal Difference: 0
Goals For: 0
Goals Against: 0

Team: BRN
Position: 4
Points: 0
Goal Difference: 0
Goals For: 0
Goals Against: 0

Team: CHE
Position: 5
Points: 0
Goal Difference: 0
Goals For: 0
Goals Against: 0

Team: CRY
Position: 6
Points: 0
Goal Difference: 0
Goals For: 0
Goals Against: 0

Team: EVE
Position: 7
Points: 0
Goal Difference: 0
Goals For: 0
Goals Against: 0

Team: FOR
Position: 8
Points: 0
Goal Difference: 0
Goals For: 0
Goals Against: 0

Team: FUL
Position: 9
Points: 0
Goal Difference: 0
Goals For: 0
Goals Against: 0

Team: LEE
Position: 10
Points: 0
Goal Difference: 0
Goals For: 0
Goals Against: 0

Team: LEI
Position: 11
Points: 0
Goal Difference: 0
Goals For: 0
Goals Against: 0

Team: LIV
Position: 12
Points: 0
Goal Difference: 0
Goals For: 0
Goals Against: 0

Team: MNU
Pos

In [16]:
league_record = pd.read_csv("league_record.csv")
league_record.head(10)

,league_id,week,hour,minutes,HT,AT,HCS,ACS,HL1S,AL1S,HL2S,AL2S,HL3S,AL3S,hl1_stat,al1_stat,hl2_stat,al2_stat,hl3_stat,al3_stat,h_wins,a_wins,h_draws,a_draws,h_loss,a_loss,h_gf,a_gf,h_ga,a_ga,h_gd,a_gd,hs_avg,as_avg,hw_rat,aw_rat,hl_rat,al_rat,hd_rat,ad_rat,ht_points,at_points,ht_pos,at_pos
0,6095,1,12,52,EVE,MNC,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,2,2,1,-1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,3,9,1
1,6095,1,12,52,BRN,TOT,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,-1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,3,7,2
2,6095,1,12,52,BRI,FUL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0,1,12
3,6095,1,12,52,LEI,WHU,2,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,2,1,1,2,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0,2,19
4,6095,1,12,52,WOL,ARS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,6,5
5,6095,1,12,52,ASV,LIV,2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,2,3,3,2,-1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,3,8,3
6,6095,1,12,52,CRY,FOR,3,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,3,1,1,3,2,-2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0,2,14
7,6095,1,12,52,CHE,MNU,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,3,3,3,3,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,8,9
8,6095,1,12,52,LEE,BOU,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,2,2,2,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,10,8
9,6095,1,12,52,SOU,NWC,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,2,2,0,-2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,3,19,6


In [21]:
league_record.columns.tolist()

['league_id',
 'week',
 'hour',
 'minutes',
 'HT',
 'AT',
 'HCS',
 'ACS',
 'HL1S',
 'AL1S',
 'HL2S',
 'AL2S',
 'HL3S',
 'AL3S',
 'hl1_stat',
 'al1_stat',
 'hl2_stat',
 'al2_stat',
 'hl3_stat',
 'al3_stat',
 'h_wins',
 'a_wins',
 'h_draws',
 'a_draws',
 'h_loss',
 'a_loss',
 'h_gf',
 'a_gf',
 'h_ga',
 'a_ga',
 'h_gd',
 'a_gd',
 'hs_avg',
 'as_avg',
 'hw_rat',
 'aw_rat',
 'hl_rat',
 'al_rat',
 'hd_rat',
 'ad_rat',
 'ht_points',
 'at_points',
 'ht_pos',
 'at_pos']